## Scenario 1: A single data scientist participating in an ML competition

MLflow setup:
* Tracking server: no
* Backend store: local filesystem
* Artifacts store: local filesystem

The experiments can be explored locally by launching the MLflow UI.

In [8]:
import mlflow

In [9]:
mlflow.set_tracking_uri("http://127.0.0.1:5001")
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5001'


In [10]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/154765283189832012', creation_time=1761711193887, experiment_id='154765283189832012', last_update_time=1761711193887, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1761614371564, experiment_id='0', last_update_time=1761614371564, lifecycle_stage='active', name='Default', tags={'mlflow.experimentKind': 'custom_model_development'}>]

### Creating an experiment and logging a new run

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/10/29 05:29:23 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/10/29 05:29:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/29 05:29:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'mlflow-artifacts:/190105288112393163/bf76e6cc0e0e4284849fe2d1c3af441e/artifacts'
🏃 View run tasteful-sheep-522 at: http://127.0.0.1:5001/#/experiments/190105288112393163/runs/bf76e6cc0e0e4284849fe2d1c3af441e
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/190105288112393163


In [12]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/190105288112393163', creation_time=1761715763762, experiment_id='190105288112393163', last_update_time=1761715763762, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/154765283189832012', creation_time=1761711193887, experiment_id='154765283189832012', last_update_time=1761711193887, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1761614371564, experiment_id='0', last_update_time=1761614371564, lifecycle_stage='active', name='Default', tags={'mlflow.experimentKind': 'custom_model_development'}>]

### Interacting with the model registry

In [13]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [14]:
from mlflow.exceptions import MlflowException

try:
    client.search_registered_models()
except MlflowException:
    print("It's not possible to access the model registry :(")